Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [2]:
labels = [{'name':'Mask', 'id':1}, {'name':'NoMask', 'id':2}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
C:\Users\mohan\AppData\Local\Temp\ipykernel_16336\66354810.py:3: SyntaxWarning: invalid escape sequence '\l'
  with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:


# 2. Create TF records

In [3]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Traceback (most recent call last):
  File "c:\Users\mohan\OneDrive\Desktop\real-sign-language\Tensorflow\scripts\generate_tfrecord.py", line 27, in <module>
    import tensorflow.compat.v1 as tf
  File "C:\Users\mohan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow\__init__.py", line 40, in <module>
    from tensorflow.python import pywrap_tensorflow as _pywrap_tensorflow  # pylint: disable=unused-import
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'tensorflow.python'
Traceback (most recent call last):
  File "c:\Users\mohan\OneDrive\Desktop\real-sign-language\Tensorflow\scripts\generate_tfrecord.py", line 27, in <module>
    import tensorflow.compat.v1 as tf
  File "C:\Users\mohan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow\__init__.py", line 40

# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
error: RPC failed; curl 92 HTTP/2 stream 5 was not closed cleanly: CANCEL (err 8)
error: 867 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [5]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [6]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [7]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

<string>:1: SyntaxWarning: invalid escape sequence '\w'
'cp' is not recognized as an internal or external command,
operable program or batch file.


# 5. Update Config For Transfer Learning

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


ModuleNotFoundError: No module named 'tensorflow.python'

In [9]:
CONFIG_PATH = "Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config"

In [10]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from tensorflow.io import gfile

CONFIG_PATH = "Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config"
def load_pipeline_config(config_path):
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

    # Read and parse the pipeline config file
    with gfile.GFile(config_path, "r") as f:
        proto_str = f.read()
        text_format.Merge(proto_str, pipeline_config)

    # Convert the parsed protobuf to a dictionary
    configs = config_util.create_configs_from_pipeline_proto(pipeline_config)
    return configs

# Load the pipeline configuration
configs = load_pipeline_config(CONFIG_PATH)

# Print the loaded configurations for debugging
print(configs)


{'model': ssd {
  num_classes: 2
  image_resizer {
    fixed_shape_resizer {
      height: 320
      width: 320
    }
  }
  feature_extractor {
    type: "ssd_mobilenet_v2_fpn_keras"
    depth_multiplier: 1.0
    min_depth: 16
    conv_hyperparams {
      regularizer {
        l2_regularizer {
          weight: 3.9999998989515007e-05
        }
      }
      initializer {
        random_normal_initializer {
          mean: 0.0
          stddev: 0.009999999776482582
        }
      }
      activation: RELU_6
      batch_norm {
        decay: 0.996999979019165
        scale: true
        epsilon: 0.0010000000474974513
      }
    }
    use_depthwise: true
    override_base_feature_extractor_hyperparams: true
    fpn {
      min_level: 3
      max_level: 7
      additional_layer_depth: 128
    }
  }
  box_coder {
    faster_rcnn_box_coder {
      y_scale: 10.0
      x_scale: 10.0
      height_scale: 5.0
      width_scale: 5.0
    }
  }
  matcher {
    argmax_matcher {
      matched_thresho

In [11]:
config

Available objects for config:
    AliasManager
    DisplayFormatter
    HistoryManager
    IPCompleter
    IPKernelApp
    LoggingMagics
    MagicsManager
    OSMagics
    PrefilterManager
    ScriptMagics
    StoreMagics
    ZMQInteractiveShell


In [12]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [13]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [14]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [15]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


# 7. Load Train Model From Checkpoint

In [16]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


In [17]:
import tensorflow as tf
from object_detection.utils import config_util
import numpy as np
import os

# Paths (Replace with your paths)
SAVED_MODEL_PATH = "Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model"  # Path to the exported model directory
CHECKPOINT_PATH = "Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint"   # Path to the checkpoint directory

# Load the saved model
detect_fn = tf.saved_model.load(SAVED_MODEL_PATH)

# Define a function for running detection
def run_inference(image):
    """
    Perform object detection on an input image.
    Args:
        image: A NumPy array representing the input image.
    Returns:
        detections: A dictionary containing the detection results.
    """
    # Convert the image to a tensor
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]  # Add batch dimension
    
    # Run inference
    detections = detect_fn(input_tensor)
    
    return detections

# Example Usage
if __name__ == "__main__":
    # Replace with a sample image as NumPy array
    sample_image = np.random.rand(640, 640, 3).astype(np.uint8)  # Placeholder image
    
    detections = run_inference(sample_image)
    print(detections)


2024-12-29 04:39:05.104393: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


{'num_detections': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([100.], dtype=float32)>, 'detection_multiclass_scores': <tf.Tensor: shape=(1, 100, 91), dtype=float32, numpy=
array([[[2.40962923e-04, 2.34315032e-03, 8.25872179e-04, ...,
         5.31191996e-04, 1.88509771e-03, 4.45755257e-04],
        [2.04591925e-04, 3.63720488e-03, 2.15955242e-03, ...,
         9.21974133e-04, 2.74931057e-03, 1.09363045e-03],
        [1.28045431e-04, 2.32595857e-03, 1.43585540e-03, ...,
         1.03297585e-03, 1.93839485e-03, 6.19258790e-04],
        ...,
        [1.92099495e-03, 2.07633688e-03, 1.26298040e-03, ...,
         8.85477813e-04, 3.06261843e-03, 1.27432519e-03],
        [2.08881993e-05, 2.00390263e-04, 1.12296526e-04, ...,
         2.25259078e-04, 8.79836734e-05, 3.06012262e-05],
        [1.04016031e-03, 1.13887656e-02, 2.14778306e-03, ...,
         3.09327291e-03, 4.77604615e-03, 6.80315634e-03]]], dtype=float32)>, 'detection_classes': <tf.Tensor: shape=(1, 100), dtype=float32, nump

# 8. Detect in Real-Time

In [18]:
import cv2 
import numpy as np

In [19]:
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import string_int_label_map_pb2

def load_label_map_custom(path):
    """Load label map from the given file path."""
    label_map = string_int_label_map_pb2.StringIntLabelMap()
    with open(path, 'r') as fid:  # Using built-in open
        text_format.Merge(fid.read(), label_map)
    return label_map

def create_category_index_from_labelmap(label_map_path, use_display_name=True):
    """Create a category index from the label map."""
    label_map = load_label_map_custom(label_map_path)
    categories = convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=use_display_name
    )
    return create_category_index(categories)

def convert_label_map_to_categories(label_map, max_num_classes, use_display_name):
    """Convert label map to categories."""
    categories = []
    for item in label_map.item:
        if not 0 < item.id <= max_num_classes:
            continue
        categories.append({
            'id': item.id,
            'name': item.display_name if use_display_name and item.HasField('display_name') else item.name
        })
    return categories

def create_category_index(categories):
    """Create category index from categories."""
    category_index = {}
    for category in categories:
        category_index[category['id']] = category
    return category_index


In [20]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

[ WARN:0@9.994] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@9.995] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [21]:
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from object_detection.protos import string_int_label_map_pb2
from google.protobuf import text_format

# Constants
SAVED_MODEL_PATH = 'Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model'  # Replace with your saved model path
ANNOTATION_PATH = 'Tensorflow/workspace/annotations'  # Replace with your annotation folder path
LABEL_MAP_FILE = f'{ANNOTATION_PATH}/label_map.pbtxt'
VIDEO_SOURCE = 0  # Use 0 for webcam or a file path for video

# Load the model
print("Loading model...")
detect_fn = tf.saved_model.load(SAVED_MODEL_PATH)
print("Model loaded.")

# Load the label map using `open`
def load_label_map_custom(path):
    """Load label map from the given file path."""
    label_map = string_int_label_map_pb2.StringIntLabelMap()
    with open(path, 'r') as fid:  # Using built-in open instead of tf.io.gfile.GFile
        text_format.Merge(fid.read(), label_map)
    return label_map

def create_category_index_from_labelmap(label_map_path, use_display_name=True):
    """Create a category index from the label map."""
    label_map = load_label_map_custom(label_map_path)
    categories = []
    for item in label_map.item:
        if not 0 < item.id <= 90:  # Limit to 90 categories (COCO dataset standard)
            continue
        categories.append({
            'id': item.id,
            'name': item.display_name if use_display_name and item.HasField('display_name') else item.name
        })
    return {category['id']: category for category in categories}

# Load the label map
print("Loading label map...")
category_index = create_category_index_from_labelmap(LABEL_MAP_FILE, use_display_name=True)
print("Label map loaded.")

# Function to process a frame and return the annotated frame
def process_frame(frame):
    # Convert to RGB (OpenCV uses BGR by default)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_np = np.array(frame_rgb, dtype=np.uint8)

    # Prepare input tensor
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, axis=0), dtype=tf.uint8)

    # Run inference
    detections = detect_fn(input_tensor)

    # Process detections
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # Visualize detections
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np,
        detections['detection_boxes'],
        detections['detection_classes'] + 1,  # Offset for label IDs
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.5,
        agnostic_mode=False
    )

    # Convert back to BGR for OpenCV display
    return cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

# Start video capture
print("Starting video capture...")
cap = cv2.VideoCapture(VIDEO_SOURCE)

if not cap.isOpened():
    print("Error: Unable to access video source.")
    exit()

try:
    while True:
        ret, frame = cap.read()

        if not ret or frame is None:
            print("Warning: Unable to read frame.")
            break

        # Process the frame
        annotated_frame = process_frame(frame)

        # Display the frame
        cv2.imshow('Object Detection', cv2.resize(annotated_frame, (800, 600)))

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Error: {e}")

finally:
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Video capture stopped and resources released.")


Loading model...


Model loaded.
Loading label map...
Label map loaded.
Starting video capture...
Error: Unable to access video source.
Video capture stopped and resources released.


[ WARN:0@17.080] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@17.080] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [ ]:
detections = detect_fn(input_tensor)